<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1">Setup</a></span></li><li><span><a href="#导入训练样本" data-toc-modified-id="导入训练样本-2">导入训练样本</a></span><ul class="toc-item"><li><span><a href="#读取数据" data-toc-modified-id="读取数据-2.1">读取数据</a></span><ul class="toc-item"><li><span><a href="#说明——样本与标签" data-toc-modified-id="说明——样本与标签-2.1.1">说明——样本与标签</a></span></li></ul></li><li><span><a href="#增加图片的通道" data-toc-modified-id="增加图片的通道-2.2">增加图片的通道</a></span><ul class="toc-item"><li><span><a href="#说明——训练图片格式" data-toc-modified-id="说明——训练图片格式-2.2.1">说明——训练图片格式</a></span></li></ul></li><li><span><a href="#将标签转换为-onehot" data-toc-modified-id="将标签转换为-onehot-2.3">将标签转换为 onehot</a></span><ul class="toc-item"><li><span><a href="#说明——onehot" data-toc-modified-id="说明——onehot-2.3.1">说明——onehot</a></span></li></ul></li></ul></li><li><span><a href="#将训练数据划分为-batch" data-toc-modified-id="将训练数据划分为-batch-3">将训练数据划分为 batch</a></span></li><li><span><a href="#定义神经网络" data-toc-modified-id="定义神经网络-4">定义神经网络</a></span><ul class="toc-item"><li><span><a href="#说明——Convolution" data-toc-modified-id="说明——Convolution-4.1">说明——Convolution</a></span></li><li><span><a href="#说明——Dense" data-toc-modified-id="说明——Dense-4.2">说明——Dense</a></span></li><li><span><a href="#说明——Maxpool" data-toc-modified-id="说明——Maxpool-4.3">说明——Maxpool</a></span></li><li><span><a href="#说明——loss、Softmax、cross-entropy" data-toc-modified-id="说明——loss、Softmax、cross-entropy-4.4">说明——loss、Softmax、cross-entropy</a></span></li><li><span><a href="#说明——优化函数与梯度下降" data-toc-modified-id="说明——优化函数与梯度下降-4.5">说明——优化函数与梯度下降</a></span></li></ul></li><li><span><a href="#开始训练" data-toc-modified-id="开始训练-5">开始训练</a></span></li><li><span><a href="#预测" data-toc-modified-id="预测-6">预测</a></span></li></ul></div>

# 30分钟从零开始搞定手写汉字识别

## Setup

In [ ]:
from PIL import Image
import numpy as np
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential
from keras.optimizers import Adam, Nadam, SGD
from keras.preprocessing import image
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras import backend as K
from skimage import exposure
from matplotlib import pyplot as plt, animation

%matplotlib inline
%precision 4
np.set_printoptions(precision=4, linewidth=200)

打印narray的便利函数

In [ ]:
def plots(ims, interp=False, titles=None):
    ims=np.array(ims)
    mn,mx=ims.min(),ims.max()
    f = plt.figure(figsize=(12,24))
    for i in range(len(ims)):
        sp=f.add_subplot(1, len(ims), i+1)
        if not titles is None: sp.set_title(titles[i], fontsize=18)
        plt.imshow(ims[i], interpolation=None if interp else 'none', vmin=mn,vmax=mx)

def plot(im, interp=False):
    f = plt.figure(figsize=(3,6), frameon=True)
    plt.imshow(im, interpolation=None if interp else 'none')

plt.gray()
plt.close()

## 导入训练样本

### 读取数据

这里读取的是MNIST数据，如果是手写汉字识别应该使用 ImageDataGenerator 类的flow_from_directory函数，具体使用非常简单，可以参看[官网教程链接](https://keras-cn.readthedocs.io/en/latest/preprocessing/image/#_1)

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#### 说明——样本与标签

查看训练样本和测试样本的shape

In [ ]:
(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

样本

In [ ]:
plots(x_train[:5])

标签

In [ ]:
y_train[:5]

样本在内存中的表示

In [ ]:
x_train[0]

### 增加图片的通道

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

#### 说明——训练图片格式

如果使用flow_from_directory函数读取数据，可以跳过这一步，输入的图片是否符合训练条件，可以通过shape确认

In [ ]:
(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

### 将标签转换为 onehot

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#### 说明——onehot

y_train 是我们的标签，在分类任务中，我们不能直接使用这样的标签，我们需要转换为 onehot 向量，什么是 onehot？其实就是将整数用向量表示，该向量的维度等于分类任务的类别数，如果是MNIST就是10，如果是HCCR就是3755，该向量只有第N个分量为1，其他都是0，N就是整数的数值

In [ ]:
y_train[:5]

## 将训练数据划分为 batch

In [ ]:
batch_size = 128
gen = image.ImageDataGenerator()
train_batchs = gen.flow(x_train, y_train, batch_size=batch_size)
test_batchs = gen.flow(x_test, y_test, batch_size=batch_size)

## 定义神经网络

In [ ]:
def get_vgg_improve():
    model = Sequential([
        Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        BatchNormalization(axis=3),
        Conv2D(32, 3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=3),
        Conv2D(64, 3, activation='relu'),
        BatchNormalization(axis=3),
        Conv2D(64, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### 说明——Convolution

简单的理解，卷积就是对图像特征的提取，有多少个卷积层就意味着提取了多少种特征，所以，如果你的训练样本非常复杂，就需要考虑增加卷积层，以提取足够的特征

输入

In [ ]:
np.arange(9).reshape(3,3)

卷积核,卷积核中的数值就是我们需要计算的权重

In [ ]:
np.arange(4).reshape(2,2)

手动卷积

In [ ]:
[[0*0 + 1*1 + 2*3 + 3*4, 0*1 + 1*2 + 2*4 + 3*5],
 [0*3 + 1*4 + 2*6 + 3*7, 0*4 + 1*5 + 2*7 + 3*8]]

### 说明——Dense

全连接层，最重要的应用在于改变输出的维度，例如，我的卷积层最后输出了3\*3的矩阵，但是我要做猫狗二分类，怎么办，就用一个Flatten层把输出1\*9的向量，然后再乘以9\*2的全连接矩阵，输出就是1\*2的向量，此时向量中的第一个分量大我们就可以认为是狗，反之则是猫

In [ ]:
np.arange(3)

这个矩阵中的值，就是我们需要训练的权重

In [ ]:
np.arange(6).reshape(3,2)

In [ ]:
[0*0 + 1*2 + 2*4, 0*1 + 1*3 + 2*5]

### 说明——Maxpool

maxpool的作用是，使计算机能够获取更大范围的图像特征，即便是人类，如果每次只能查看9个像素的内容，也很难猜出当前图片是什么内容，所以需要通过maxpool降低图像特征提取的细节，提升图像特征提取范围

In [ ]:
np.arange(16).reshape(4,4)

In [ ]:
[[np.max([0, 1, 4, 5]), np.max([2, 3, 6, 7])],
 [np.max([8, 9, 12, 13]), np.max([10, 11, 14, 15])]]

### 说明——loss、Softmax、cross-entropy

* loss 即损失函数，标签与模型的输出的差异就是loss，在MNIST任务中，标签和模型输出都是10维向量，loss就是两个向量的差异，我们用交叉熵衡量这个差异。
* cross-entropy 不好理解，我们可以简单的理解为两个向量的方差，为什么不选平方差而选择交叉熵，下面是y=x*x的函数图，可以看到在接近0的位置也就是我们模型拟合最好的位置附近，导数非常接近0，如果此时采用梯度下降则会收敛的非常慢。交叉熵就是为了克服这个问题，交叉熵损失函数具有平方差一样的特性，非0，同时在0附近的梯度很陡峭。
* softmax 神经网络输出没办法告诉你结果是1还是0，只能告诉你1的概率是多少，0的概率是多少。直接用神经网路模型输出计算交叉熵时有个问题，数值得绝对大小并不能表示结果的概率大小，例如我们的标签是[1, 0] 模型的输出是[18,0],不说交叉熵，我们按平方差来算，也会认为我们的模型预测的非常不好，所以为了让输出的各个维度准确的表示概率，我们需要将结果先做softmax，softmax处理过后，向量的各个分类之和等于1，此时分量的大小可以准确的表达概率大小，就可以做交叉熵了。在预测结果的时候，我们会调用argmax获取概率最大的标签。

In [ ]:
x = np.arange(100) - 50
y = x * x
plt.plot(x, y)

### 说明——优化函数与梯度下降

优化函数就是根据loss的值不停地执行梯度下降算法，这里通过梯度下降求解一元一次方程，以说明梯度下降算法

假设，我们的目标函数为 ``y = 3 * x + 8``, 这里的 w=3 和 b=8 就是我之前反复提到的权重值, 机器学习需要训练数据，这里我们很容易自己创造

In [ ]:
def lin(w, b, x): return w*x+b
n = 30
x = np.random.rand(n)
y = lin(3, 8, x)
(x, y)

In [ ]:
plt.scatter(x, y)

训练开始前，我们不知道 w = 3， b = 8，所以我们一般会给一个随机的初始值

In [ ]:
w = 1
b = -1

In [ ]:
fig = plt.figure(dpi=100, figsize=(5, 4))
plt.xticks(np.arange(0, 1, 0.1))  
plt.yticks(np.arange(0, 15, 1)) 
plt.scatter(x,y)
plt.plot(x,lin(w, b, x));

交叉熵太复杂，loss用方差，所以相当于把每个``x``分别带入``3*x+8``与``10*x-1``然后求方差

In [ ]:
def sse(y,y_pred): return ((y-y_pred)**2).sum()
def loss(y,w,b,x): return sse(y, lin(w, b, x))
def avg_loss(y, w, b,x): return np.sqrt(loss(y, w, b,x) / n)

In [ ]:
avg_loss(y, w, b, x)

In [ ]:
def upd():
    global w, b
    y_pred = lin(w, b, x)
    dydb = 2 * (y_pred - y)
    dyda = x*dydb
    w -= lr*dyda.mean()
    b -= lr*dydb.mean()

In [ ]:
lr=0.01
# d[(y-(a*x+b))**2,b] = 2 (b + a x - y)      = 2 (y_pred - y)
# d[(y-(a*x+b))**2,a] = 2 x (b + a x - y)    = x * dy/db

In [ ]:
for _ in np.arange(10):
    upd()
fig = plt.figure(dpi=100, figsize=(5, 4))
plt.scatter(x,y)
plt.plot(x,lin(w, b, x));

## 开始训练

In [ ]:
model = get_vgg_improve()
model.fit_generator(train_batchs, steps_per_epoch=train_batchs.n//batch_size, epochs=4, validation_data=test_batches, validation_steps=test_batches.n//batch_size)

## 预测

In [ ]:
model = sigle_dense_model()
model.load_weights('/aiml/dfs/checkpoint/sigle_dense/v.02/weights.hdf5')
model.evaluate_generator(test_gen)
len(model.predict_generator(test_gen))